#  RCV1 Text Classification Baseline

This script implements a strong baseline for the Reuters Corpus Volume I (RCV1) 
multi-label text classification task. The dataset contains over 800,000 news stories 
with multiple labels in three taxonomies: topics, industries, and regions.

We'll focus on creating an efficient and high-performing baseline using modern approaches 
in scikit-learn and transformers.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_rcv1
from tqdm import tqdm

# Transformer-related imports
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.nn import BCEWithLogitsLoss
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

def load_and_explore_dataset():
    """Load and explore the RCV1 dataset."""
    print('Loading RCV1 dataset from scikit-learn...')
    rcv1 = fetch_rcv1()
    
    # Convert sparse matrix to dense for text representation
    # We'll create text by using word indices and their TF-IDF values
    texts = []
    
    print('Converting feature vectors to text...')
    for i in tqdm(range(rcv1.data.shape[0])):
        # Get non-zero features for this document
        doc_features = rcv1.data[i].toarray().flatten()
        non_zero_indices = np.where(doc_features > 0)[0]
        
        # Create text by using word indices and their TF-IDF values
        text = []
        for idx in non_zero_indices:
            # Create a word-like token from the index
            word = f"word_{idx}"
            # Repeat based on TF-IDF value (rounded to integer)
            count = int(doc_features[idx] * 10)  # Scale up to get more repetitions
            text.extend([word] * count)
        
        texts.append(' '.join(text))
    
    # Convert to numpy array
    texts = np.array(texts)
    
    # Get labels
    labels = rcv1.target.toarray()
    
    print('\nDataset Features:')
    print(f'Number of samples: {len(texts)}')
    print(f'Number of labels: {labels.shape[1]}')
    
    # Show a sample
    print('\nSample document:')
    print(f'Text length: {len(texts[0])}')
    print(f'Labels: {labels[0]}')
    
    return texts, labels

def split_dataset(texts, labels):
    """Split the dataset into train, validation and test sets."""
    # Use the first 23,149 documents as training set (as per LYRL2004 split)
    train_size = 23149
    
    # Split the data
    X_train = texts[:train_size]
    X_test = texts[train_size:]
    y_train = labels[:train_size]
    y_test = labels[train_size:]
    
    # Create a validation set from training data
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train,
        test_size=0.1,
        random_state=42
    )
    
    print('Dataset splits:')
    print(f'Training set: {len(X_train)} samples')
    print(f'Validation set: {len(X_val)} samples')
    print(f'Test set: {len(X_test)} samples')
    
    return X_train, X_val, X_test, y_train, y_val, y_test

class BERTForMultiLabelClassification(nn.Module):
    """BERT/DeBERTa model for multi-label classification."""
    def __init__(self, num_labels, model_name='microsoft/deberta-v3-base'):
        super().__init__()
        self.num_labels = num_labels
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            problem_type="multi_label_classification"
        )
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return outputs

class RCV1Dataset(Dataset):
    """Dataset class for RCV1."""
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.FloatTensor(labels)
        }

def train_epoch(model, dataloader, optimizer, scheduler, criterion, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc='Training')
    
    for batch in progress_bar:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    
    return total_loss / len(dataloader)

def evaluate_bert_predictions(model, dataloader, device):
    """Evaluate BERT model predictions."""
    model.eval()
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].numpy()
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.sigmoid(outputs.logits).cpu().numpy() > 0.5
            
            all_labels.extend(labels)
            all_predictions.extend(predictions)
    
    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    
    metrics = {
        'Micro F1': f1_score(all_labels, all_predictions, average='micro'),
        'Macro F1': f1_score(all_labels, all_predictions, average='macro'),
        'Weighted F1': f1_score(all_labels, all_predictions, average='weighted'),
        'Samples F1': f1_score(all_labels, all_predictions, average='samples'),
        'Micro Precision': precision_score(all_labels, all_predictions, average='micro'),
        'Micro Recall': recall_score(all_labels, all_predictions, average='micro')
    }
    
    return metrics

def visualize_results(val_metrics, test_metrics):
    """Visualize model performance metrics."""
    # Create a comparison plot
    metrics_df = pd.DataFrame({
        'Validation': val_metrics,
        'Test': test_metrics
    }).reset_index()
    metrics_df.columns = ['Metric', 'Validation', 'Test']
    
    # Melt the dataframe for easier plotting
    melted_df = pd.melt(metrics_df, id_vars=['Metric'], var_name='Dataset', value_name='Score')
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    sns.barplot(data=melted_df, x='Metric', y='Score', hue='Dataset')
    plt.xticks(rotation=45)
    plt.title('BERT Model Performance Metrics')
    plt.tight_layout()
    plt.show()

def main():
    """Main function to run the entire pipeline."""
    # Load and explore dataset
    texts, labels = load_and_explore_dataset()
    
    # Split dataset
    X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(texts, labels)
    
    # Initialize BERT/DeBERTa model
    print('Loading DeBERTa tokenizer and model...')
    tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
    model = BERTForMultiLabelClassification(
        num_labels=labels.shape[1]
    ).to(device)
    
    # Create datasets
    train_dataset = RCV1Dataset(X_train, y_train, tokenizer)
    val_dataset = RCV1Dataset(X_val, y_val, tokenizer)
    
    # Create dataloaders
    batch_size = 8
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )
    
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )
    
    # Training settings
    num_epochs = 3
    num_training_steps = num_epochs * len(train_dataloader)
    num_warmup_steps = num_training_steps // 10
    
    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )
    criterion = BCEWithLogitsLoss()
    
    # Train BERT model
    print('Training DeBERTa model...')
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, criterion, device)
        print(f'Average training loss: {train_loss:.4f}')
    
    # Evaluate model
    print('\nEvaluating BERT model...')
    
    # Evaluate on validation set
    val_metrics = evaluate_bert_predictions(model, val_dataloader, device)
    print('\nValidation Metrics:')
    for metric_name, value in val_metrics.items():
        print(f'{metric_name}: {value:.4f}')
    
    # Create test dataset and dataloader
    test_dataset = RCV1Dataset(X_test, y_test, tokenizer)
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False
    )
    
    # Evaluate on test set
    test_metrics = evaluate_bert_predictions(model, test_dataloader, device)
    print('\nTest Metrics:')
    for metric_name, value in test_metrics.items():
        print(f'{metric_name}: {value:.4f}')
    
    # Visualize results
    visualize_results(val_metrics, test_metrics)

if __name__ == '__main__':
    main() 